In [1]:
import pandas as pd
import os
import requests
from pydracor import *

In [2]:
german_corpus = Corpus('ger')
german_plays = german_corpus.play_ids()

if not os.path.exists('data/relations'):
    os.makedirs('data/relations')

if not os.path.exists('data/cooccurence'):
    os.makedirs('data/cooccurence')

for play_id in german_plays:
    play = Play(play_id)
    filename = f"data/relations/{play_id}.graphml"
    try:
        relations_graphml = play.relations_graphml()
    except requests.HTTPError:
        continue
    # networkX doesn't support mix of directed+undirected Graphs & nx_altair's arrows look broken
    # workaround: make graph undirected
    relations_graphml = relations_graphml.replace('directed="true"', 'directed="false"')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(relations_graphml)
    filename = f"data/cooccurence/{play_id}.graphml"
    cooccurence_graphml = play.graphml()
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(cooccurence_graphml)

In [2]:
german_corpus = Corpus('ger')
german_metadata = pd.DataFrame(german_corpus.metadata())
existing_relations_files = os.listdir('data/relations')
existing_relations_play_ids = [filename.split('.')[0] for filename in existing_relations_files]
german_metadata_filtered = german_metadata[german_metadata['id'].isin(existing_relations_play_ids)]
german_metadata_filtered.to_csv('data/german_metadata.csv', index=False, encoding='utf-8')